
# Python для анализа данных
## Домашнее задание 6

* Сгрузить с помощью библиотек requests и BeautifulSoup данные с html-страницы. В каком формате сохранять данные на предварительном этапе (список списков, кортежей, словарь, совершенно не важно).
* Сохранить сгруженные данные в формате датафрейма pandas. Присвоить столбцам таблицы внятные названия. При необходимости преобразовать типы данных в столбцах (чтобы числовые данные имели формат float или integer). Если в таблице присутствует «мусор», отфильтровать ненужные строки.

### Вариант 1 (на 10 – сложный)
Результаты президентских выборов 2018 года в Бурятии. Ссылка на страницу. Каждая строка в итоговом датафрейме должна соответствовать одной территориальной избирательной комиссии (одному району), по которой представлены все 20 показателей (проценты за кандидатов не нужны, достаточно абсолютного числа голосов). В таблице должен быть 21 столбец (название ТИК и 20 показателей).

In [21]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [22]:
url = "http://www.vybory.izbirkom.ru/region/region/izbirkom?action=show&root=1000004&tvd=100100084849070&vrn=100100084849062&region=0&global=true&sub_region=0&prver=0&pronetvd=null&vibid=100100084849070&type=227"
page = requests.get(url)

In [23]:
soup = BeautifulSoup(page.text, 'lxml')

In [24]:
urls = []       # тут хранятся ссылки на результаты голосования по каждой избирательной комиссии
name_TIK = []   # тут хранятся названия территориальной избирательной комиссии
for link in soup.find_all('a'):
    if '/region/region/' in link.get('href'):
        name_TIK.append(link.text)
        urls.append(link.get('href'))


In [25]:
url0 = urls[0]     
page0 = requests.get(url0)
soup0 = BeautifulSoup(page0.text, "lxml")    # html код первой страницы

In [26]:
def result_TIK(url_i):
    page_i = requests.get(url_i)
    soup_i = BeautifulSoup(page_i.text, "lxml")
    nobr_tag = soup_i.find_all("nobr")
    numbers = []      # результаты по выбранной избирательной комиссии
    i = 0
    while i < len(nobr_tag):
        if nobr_tag[i].text.isdigit():             # считаю, что первое встретившееся число -- это порядковый номер строки
            numbers.append(nobr_tag[i+2].text)     # через один -- сумма (результат голосования)
            i += 3
        else:
            i += 1
    return numbers

In [27]:
def name_cols(url_i):
    page_i = requests.get(url_i)
    soup_i = BeautifulSoup(page_i.text, "lxml")
    nobr_tag = soup_i.find_all("nobr")    
    name_col = []    # названия столбцов искомой таблицы
    i = 0
    while i < len(nobr_tag):
        if nobr_tag[i].text.isdigit():             # считаю, что первое встретившееся число -- это порядковый номер строки
            name_col.append(nobr_tag[i+1].text.strip())   # следующий за ним -- название строки
            i += 3
        else:
            i += 1
    return name_col

In [28]:
all_names_cols = ["Название ТИК"] + name_cols(urls[0])

In [29]:
def data_region(i):   # получаю список, первый элемент которого название ТИК, далее -- данные по ней
    res = []
    res.append(name_TIK[i])
    res.extend(result_TIK(urls[i]))
    return res

In [30]:
all_data = []                        # список списков конечных данных с названием ТИК
for i in range(len(name_TIK)):
    all_data.append(data_region(i))


In [31]:
df = pd.DataFrame(all_data) #, index = name_TIK)
for i in range(1, len(all_names_cols)):   # меняю тип чисел на int
    df[i] = [int(j) for j in df[i]]

df.columns = all_names_cols.copy()  # переименование столбцов


In [32]:
df

,Название ТИК,"Число избирателей, включенных в список избирателей","Число избирательных бюллетеней, полученных участковой избирательной комиссией","Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно","Число избирательных бюллетеней, выданных в помещении для голосования в день голосования","Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования",Число погашенных избирательных бюллетеней,Число избирательных бюллетеней в переносных ящиках для голосования,Число бюллетеней в стационарных ящиках для голосования,Число недействительных избирательных бюллетеней,...,Число утраченных избирательных бюллетеней,"Число избирательных бюллетеней, не учтенных при получении",Бабурин Сергей Николаевич,Грудинин Павел Николаевич,Жириновский Владимир Вольфович,Путин Владимир Владимирович,Собчак Ксения Анатольевна,Сурайкин Максим Александрович,Титов Борис Юрьевич,Явлинский Григорий Алексеевич
0,Баргузинская,16488,16385,2,9314,963,6106,965,9314,169,...,0,0,36,1245,679,7980,89,33,21,27
1,Баунтовская,6883,6724,797,3836,713,1378,1510,3836,57,...,0,0,29,578,479,4072,72,25,14,20
2,Бичурская,16020,13340,0,9637,1527,2176,1527,9637,97,...,0,0,56,1966,1088,7757,82,68,33,17
3,Джидинская,15631,13420,0,10699,647,2074,647,10699,113,...,0,0,30,1289,623,9083,105,57,32,14
4,Еравнинская,10371,11398,107,9058,323,1910,430,9055,121,...,0,0,27,903,470,7739,132,44,26,23
5,Заиграевская,33025,30000,0,22323,2072,5605,2072,22323,327,...,0,0,95,3114,1864,18459,233,140,73,90
6,Закаменская,18407,19262,2114,11579,1645,3924,3759,11577,116,...,0,0,25,1580,518,12869,134,35,21,38
7,Иволгинская,29583,24966,0,19956,896,4114,896,19954,278,...,0,0,70,3256,891,15537,487,101,75,155
8,Кабанская,38660,32496,0,24575,2135,5786,2135,24562,546,...,0,0,116,4231,2656,18580,208,149,111,100
9,Кижингинская,11389,11645,0,10225,509,911,509,10225,176,...,0,0,25,681,226,9479,74,24,25,24


In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25 entries, 0 to 24
Data columns (total 21 columns):
Название ТИК                                                                                 25 non-null object
Число избирателей, включенных в список избирателей                                           25 non-null int64
Число избирательных бюллетеней, полученных участковой избирательной комиссией                25 non-null int64
Число избирательных бюллетеней, выданных избирателям, проголосовавшим досрочно               25 non-null int64
Число избирательных бюллетеней, выданных в помещении для голосования в день голосования      25 non-null int64
Число избирательных бюллетеней, выданных вне помещения для голосования в день голосования    25 non-null int64
Число погашенных избирательных бюллетеней                                                    25 non-null int64
Число избирательных бюллетеней в переносных ящиках для голосования                           25 non-null int64
Число бю

In [34]:
df.to_csv("html-Gorlenko.csv", index=False)